In [2]:
import logging, matplotlib, os, sys, glob
import scanpy as sc
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import colors
import pandas as pd
from glbase3 import genelist

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 9.06 µs


In [4]:
plt.rcParams['figure.figsize']=(4,4)
sc.settings.verbosity = 3
sc.set_figure_params(dpi=200, dpi_save=200)
matplotlib.rcParams['pdf.fonttype']=42
matplotlib.rcParams['font.size']=10

In [5]:
from glbase3 import genelist, glload

In [6]:
sc.settings.figdir = 'diffexp-tes'

In [7]:
[os.remove(f) for f in glob.glob('{0}/*.pdf'.format(sc.settings.figdir))]
[os.remove(f) for f in glob.glob('gls/*.glb'.format(sc.settings.figdir))]
[os.remove(f) for f in glob.glob('gls/*.tsv'.format(sc.settings.figdir))]

[None, None, None, None, None]

In [8]:
de_leiden = 'leiden_r0.10'

In [12]:
%%time

adata = sc.read('./de.h5ad')
TEs = set(genelist(filename='./id_hg38/TE_genes_id.hg38.txt', format={'name': 0, 'force_tsv': True})['name'])

INFO    : genelist: loaded './id_hg38/TE_genes_id.hg38.txt' found 1,045 items


CPU times: user 1.56 s, sys: 3.88 s, total: 5.44 s
Wall time: 5.43 s


In [13]:
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=True, show=False, save='genes-top25.pdf')
sc.pl.rank_genes_groups(adata, key='rank_genes_groups', show=False, save='genes.pdf')
sc.pl.rank_genes_groups_dotplot(adata, key='rank_genes_groups', show=False, save='genes-top25.pdf')

    using 'X_pca' with n_pcs = 20
Storing dendrogram info using `.uns['dendrogram_leiden_r0.10']`


{'mainplot_ax': <AxesSubplot:>,
 'group_extra_ax': <AxesSubplot:>,
 'gene_group_ax': <AxesSubplot:>,
 'size_legend_ax': <AxesSubplot:title={'center':'Fraction of cells\nin group (%)'}>,
 'color_legend_ax': <AxesSubplot:title={'center':'Mean expression\nin group'}>}

In [14]:
#print(pd.DataFrame(adata.uns['rank_genes_groups']))
print(pd.DataFrame(adata.uns['rank_genes_groups']['names']))

               0           1            2        3          4
0            DCN       CRIP1          B2M    TNNI3    ZFAND2A
1           RBP1  AL928654.3         CD74  C4orf48  KCNQ1-AS1
2         RPS15A       TIMP3        HLA-B     BEX1      TUBB8
3          RPS24       ADIRF        IFI27  ATP5IF1     REC114
4          RPLP1   ADIRF-AS1          VWF   ISYNA1      PDCD5
...          ...         ...          ...      ...        ...
2995   THE1D-int  AC093908.1         DVL1  RSPH10B     TWIST1
2996   MIR3936HG     TMEM256   AC006460.2   RNF111    LTR16B1
2997        ADNP        FCMR      LTR16A2   ANXA2R     ARRDC1
2998  MER101-int       NEDD9  HERVK13-int    AP3S2      DUSP7
2999      SPATS2       NAA20       ARFIP1   SPOCK1       AMFR

[3000 rows x 5 columns]


In [16]:
print()
topall = pd.DataFrame(adata.uns['rank_genes_groups']['names']) # get all;
fcs = pd.DataFrame(adata.uns['rank_genes_groups']['logfoldchanges'])
padj = pd.DataFrame(adata.uns['rank_genes_groups']['pvals_adj'])

In [17]:
%%time

# Matrix of DE genes:

groups = list(topall.columns.values)

newcols = {}

for group in groups:
    newcols[group] = []

    t = zip([i[group] for i in adata.uns['rank_genes_groups']['names']], [i[group] for i in adata.uns['rank_genes_groups']['logfoldchanges']], [i[group] for i in adata.uns['rank_genes_groups']['pvals_adj']])

    print('Group: {0}'.format(group))
    print(t)

    for item in t:
        if item[0] not in TEs:
            continue

        if item[1] < 2: # fold change
            continue
        if item[2] > 0.01: # just in case
            continue

        newcols[group].append({'name': item[0], 'log2FC': item[1], 'q': item[2]})

Group: 0
Group: 1
Group: 2
Group: 3
Group: 4
CPU times: user 15.6 ms, sys: 0 ns, total: 15.6 ms
Wall time: 22.6 ms


In [18]:
%%time

# join all and draw a dotplot:
for group in newcols:
    print(newcols[group])
    if newcols[group]:
        gl = genelist()
        gl.load_list(newcols[group])
        gl.saveTSV('gls/de_genes-grp{0}.tsv'.format(group))
        gl.save('gls/de_genes-grp{0}.glb'.format(group))

        genes = [i['name'] for i in newcols[group]]
        sc.pl.dotplot(adata, genes, groupby=de_leiden, dot_max=0.7, dendrogram=True, standard_scale='var', show=False, save='de-grp{0}.pdf'.format(group))
        sc.pl.matrixplot(adata, genes, groupby=de_leiden, dendrogram=True, standard_scale='var', show=False, save='de-grp{0}.pdf'.format(group))

INFO    : Saved 'gls/de_genes-grp1.tsv'
INFO    : Saved binary version of list: 'gls/de_genes-grp1.glb'


[]
[{'name': 'LTR102_Mam', 'log2FC': 2.050619, 'q': 3.5328417444668933e-17}, {'name': 'LTR28B', 'log2FC': 2.3107855, 'q': 2.089890921237523e-15}, {'name': 'Tigger17', 'log2FC': 2.2400963, 'q': 6.604206453355973e-07}]


INFO    : Saved 'gls/de_genes-grp2.tsv'
INFO    : Saved binary version of list: 'gls/de_genes-grp2.glb'


[{'name': 'Charlie23a', 'log2FC': 2.621313, 'q': 2.449818669564243e-117}, {'name': 'MLT1A1-int', 'log2FC': 2.306676, 'q': 1.4716085404571382e-50}, {'name': 'LTR33A', 'log2FC': 2.091343, 'q': 2.837475446903397e-41}, {'name': 'MLT1I-int', 'log2FC': 3.342258, 'q': 1.1670666448443836e-37}, {'name': 'ERV24B_Prim-int', 'log2FC': 2.646527, 'q': 3.55436756366937e-37}, {'name': 'X7C_LINE', 'log2FC': 2.120425, 'q': 1.5299103316410409e-34}, {'name': 'LTR86A2', 'log2FC': 3.1019907, 'q': 8.55266559171409e-27}, {'name': 'FordPrefect', 'log2FC': 2.5548985, 'q': 2.1181545036938892e-14}, {'name': 'Tigger1a_Mars', 'log2FC': 3.3383632, 'q': 5.65291173945753e-14}, {'name': 'MLT1G3-int', 'log2FC': 2.9661183, 'q': 1.2791773692336003e-12}, {'name': 'MER51D', 'log2FC': 2.511681, 'q': 7.526234885366334e-10}, {'name': 'X1_LINE', 'log2FC': 3.2345726, 'q': 3.388161937910282e-08}, {'name': 'PABL_B', 'log2FC': 2.0226207, 'q': 2.7560074607239543e-06}, {'name': 'MER70-int', 'log2FC': 2.0514367, 'q': 3.159298424807913

INFO    : Saved 'gls/de_genes-grp3.tsv'
INFO    : Saved binary version of list: 'gls/de_genes-grp3.glb'


[{'name': 'HERV15-int', 'log2FC': 2.8095365, 'q': 1.0944583073452498e-46}, {'name': 'LTR2C', 'log2FC': 3.0202823, 'q': 2.6222580689603232e-17}, {'name': 'LTR15', 'log2FC': 2.7929335, 'q': 2.2196779489475426e-15}, {'name': 'HERVS71-int', 'log2FC': 2.4223995, 'q': 6.049780551546909e-12}, {'name': 'HERVIP10F-int', 'log2FC': 2.6117108, 'q': 3.298086418242125e-09}, {'name': 'L1M3b', 'log2FC': 2.4953363, 'q': 1.0282608975420468e-05}, {'name': 'LTR66', 'log2FC': 2.1698077, 'q': 2.1826232524200138e-05}, {'name': 'LTR18B', 'log2FC': 2.5408506, 'q': 6.388462213064159e-05}, {'name': 'MER83A-int', 'log2FC': 2.6913567, 'q': 0.00021303964929752294}, {'name': 'LTR19C', 'log2FC': 2.5895913, 'q': 0.001204482627995898}]


INFO    : Saved 'gls/de_genes-grp4.tsv'
INFO    : Saved binary version of list: 'gls/de_genes-grp4.glb'


[{'name': 'LTR12C', 'log2FC': 5.66498, 'q': 5.651161908556525e-17}, {'name': 'THE1D', 'log2FC': 3.3786583, 'q': 2.7075483627627077e-10}, {'name': 'LTR12_', 'log2FC': 5.545392, 'q': 1.9431556988741745e-09}, {'name': 'LTR7', 'log2FC': 3.7253416, 'q': 1.8778669198604374e-10}, {'name': 'LTR38C', 'log2FC': 6.1935816, 'q': 3.4515502838018703e-06}, {'name': 'LTR7Y', 'log2FC': 4.8469577, 'q': 2.442841806959596e-06}, {'name': 'HERV9N-int', 'log2FC': 4.966368, 'q': 1.5892913207387036e-05}, {'name': 'MER51E', 'log2FC': 4.1631765, 'q': 9.162231682484156e-06}, {'name': 'LTR48B', 'log2FC': 3.9006326, 'q': 9.202889729868033e-06}, {'name': 'MLT1G1', 'log2FC': 2.7936726, 'q': 7.897739311484363e-06}, {'name': 'LTR12', 'log2FC': 4.591712, 'q': 6.084045339873296e-05}, {'name': 'AmnSINE1', 'log2FC': 3.5015008, 'q': 3.6852969059577224e-05}, {'name': 'LTR1B', 'log2FC': 6.035235, 'q': 0.0004936282272874955}, {'name': 'LTR7C', 'log2FC': 3.2680578, 'q': 0.0002581401357805106}, {'name': 'MER57-int', 'log2FC': 2.

In [20]:
%%time

for grp in newcols:
    if not newcols[grp]:
        continue
    for k in newcols[grp]:
        title = k['name']
        sc.pl.umap(adata, color=k['name'], size=20, legend_loc='on data',
            title=title,
            vmin=0, vmax=3,
            show=False, save='-markers-grp{0}-{1}.pdf'.format(grp, k['name']))
        #sc.pl.violin(adata, [k], groupby='disease', size=0, log=False, cut=0, show=False, save='markers-{0}-disease.pdf'.format(k))
        #sc.pl.violin(adata, [k], groupby='cell_type', size=0, log=False, cut=0, show=False, save='markers-{0}-cell_type.pdf'.format(k))

CPU times: user 21.8 s, sys: 1.41 s, total: 23.2 s
Wall time: 23.3 s
